# GPT Action Library: Outlook

このライブラリは、GPTアクションを使用してOutlookと統合するためのリソースとガイドを提供します。Outlookの機能にアクセスし、メール管理、カレンダー操作、連絡先管理などを自動化できます。

## 主な機能

- **メール管理**: メールの送信、受信、検索、整理
- **カレンダー統合**: 予定の作成、更新、削除、検索
- **連絡先管理**: 連絡先の追加、編集、検索
- **タスク管理**: To-Doリストとタスクの管理

## 前提条件

- Microsoft 365またはOutlook.comアカウント
- 適切なAPI権限とアクセストークン
- GPTアクションの設定知識

## セットアップ

1. Microsoft Graph APIへの登録
2. 必要な権限の設定
3. 認証フローの実装
4. GPTアクションの設定

このライブラリを使用することで、Outlookの強力な機能をGPTアクションに統合し、生産性を向上させることができます。

## はじめに

このページでは、特定のアプリケーション向けのGPT Actionを構築する開発者向けの手順とガイドを提供します。進める前に、まず以下の情報をよく理解しておいてください：
- [GPT Actionsの概要](https://platform.openai.com/docs/actions)
- [GPT Actions Libraryの概要](https://platform.openai.com/docs/actions/actions-library)
- [GPT Actionをゼロから構築する例](https://platform.openai.com/docs/actions/getting-started)

この特定のGPT Actionは、Microsoftのメールとカレンダーイベント用のWebサービスであるOutlookに接続する方法の概要を提供します。このアクションはユーザーのコンテキストを前提とし、Outlookからメールやカレンダーイベントの送信と取得を可能にします。

### 価値 + ビジネス活用事例

**価値**: ユーザーはChatGPTの自然言語機能を活用して、Outlookに直接接続できるようになりました

**使用例**: 
- ユーザーはその日のすべてのミーティングを検索し、ChatGPTにその日の予定を要約してもらうことができます
- ユーザーはChatGPTの出力を直接誰かにメールで送信することができます

## アプリケーション情報

### アプリケーションキーリンク

アプリケーションを開始する前に、以下のリンクを確認してください：
- アプリケーションWebサイト: https://portal.azure.com/
- アプリケーションAPI ドキュメント: https://learn.microsoft.com/en-us/graph/api/overview?view=graph-rest-1.0

### アプリケーションの前提条件

開始する前に、アプリケーション環境で以下の手順を実行していることを確認してください：
- [AzureでApp Registrationを設定する](https://portal.azure.com/?feature.tokencaching=true&feature.internalgraphapiversion=true#view/Microsoft_AAD_RegisteredApps/ApplicationsListBlade)ためのアクセス権限があることを確認する

## ChatGPT ステップ

### カスタムGPT指示

Custom GPTを作成したら、以下のテキストをInstructionsパネルにコピーしてください。質問がありますか？この手順の詳細については、[Getting Started Example](https://platform.openai.com/docs/actions/getting-started)をご確認ください。

In [ ]:
**Context**: you are specialized GPT designed to manage emails and calendar events through API connections to Microsoft Outlook. This GPT can create, read, send, and alter emails and calendar events based on user instructions. It ensures efficient handling of communication and scheduling needs by leveraging Microsoft Graph API for seamless integration with Outlook services.

**Instructions**:
- When asked to perform a task, use the available actions via the microsoft.graph.com API.
- You should behave professionally and provide clear, concise responses.
- Offer assistance with tasks such as drafting emails, scheduling meetings, organising calendar events, and retrieving email or event details.
- Ask for clarification when needed to ensure accuracy and completeness in fulfilling user requests.
- Always conclude an email by signing off with logged in user's name which can be retrieved via the User.Read endpoint


### OpenAPI スキーマ

Custom GPTを作成したら、以下のテキストをActionsパネルにコピーしてください。ご質問がありますか？この手順の詳細については、[Getting Started Example](https://platform.openai.com/docs/actions/getting-started)をご確認ください。

In [ ]:
openapi: 3.1.0
info:
  title: Microsoft Graph API Integration
  version: 1.0.0
servers:
  - url: https://graph.microsoft.com/v1.0
components:
  securitySchemes:
    OAuth2:
      type: oauth2
      flows:
        clientCredentials:
          tokenUrl: https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token
          scopes:
            https://graph.microsoft.com/User.Read: Access current user profile
            https://graph.microsoft.com/Mail.Read: Read user mail
            https://graph.microsoft.com/Mail.Send: Send mail
            https://graph.microsoft.com/Calendars.ReadWrite: Read and write user calendars
  schemas:
    UserProfile:
      type: object
      properties:
        id:
          type: string
        displayName:
          type: string
        mail:
          type: string
    UserMessage:
      type: object
      properties:
        id:
          type: string
        subject:
          type: string
        bodyPreview:
          type: string
    CalendarEvent:
      type: object
      properties:
        id:
          type: string
        subject:
          type: string
        start:
          type: object
          properties:
            dateTime:
              type: string
            timeZone:
              type: string
        end:
          type: object
          properties:
            dateTime:
              type: string
            timeZone:
              type: string
    NewEvent:
      type: object
      properties:
        subject:
          type: string
        start:
          type: object
          properties:
            dateTime:
              type: string
            timeZone:
              type: string
        end:
          type: object
          properties:
            dateTime:
              type: string
            timeZone:
              type: string
        attendees:
          type: array
          items:
            type: object
            properties:
              emailAddress:
                type: object
                properties:
                  address:
                    type: string
                  name:
                    type: string
    SendMailRequest:
      type: object
      properties:
        message:
          type: object
          properties:
            subject:
              type: string
            body:
              type: object
              properties:
                contentType:
                  type: string
                content:
                  type: string
            toRecipients:
              type: array
              items:
                type: object
                properties:
                  emailAddress:
                    type: object
                    properties:
                      address:
                        type: string
security:
  - OAuth2: []
paths:
  /me:
    get:
      operationId: getUserProfile
      summary: Get the authenticated user's profile
      security:
        - OAuth2: []
      responses:
        '200':
          description: A user profile
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/UserProfile'
  /me/messages:
    get:
      operationId: getUserMessages
      summary: Get the authenticated user's messages
      security:
        - OAuth2: []
      parameters:
        - name: $top
          in: query
          required: false
          schema:
            type: integer
            default: 10
            description: Number of messages to return
        - name: $filter
          in: query
          required: false
          schema:
            type: string
            description: OData filter query to narrow results
        - name: $orderby
          in: query
          required: false
          schema:
            type: string
            description: OData order by query to sort results
      responses:
        '200':
          description: A list of user messages
          content:
            application/json:
              schema:
                type: array
                items:
                  $ref: '#/components/schemas/UserMessage'
  /me/sendMail:
    post:
      operationId: sendUserMail
      summary: Send an email as the authenticated user
      security:
        - OAuth2: []
      requestBody:
        required: true
        content:
          application/json:
            schema:
              $ref: '#/components/schemas/SendMailRequest'
      responses:
        '202':
          description: Accepted
  /me/events:
    get:
      operationId: getUserCalendarEvents
      summary: Get the authenticated user's calendar events
      security:
        - OAuth2: []
      responses:
        '200':
          description: A list of calendar events
          content:
            application/json:
              schema:
                type: array
                items:
                  $ref: '#/components/schemas/CalendarEvent'
    post:
      operationId: createUserCalendarEvent
      summary: Create a new calendar event for the authenticated user
      security:
        - OAuth2: []
      requestBody:
        required: true
        content:
          application/json:
            schema:
              $ref: '#/components/schemas/NewEvent'
      responses:
        '201':
          description: Created
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/CalendarEvent'



## 認証手順

以下は、Outlookでの認証設定に関する手順です。ご質問がありますか？この手順の詳細については、[Getting Started Example](https://platform.openai.com/docs/actions/getting-started)をご確認ください。

### Azure手順

1. <b>アプリ登録</b>: 最初のステップは、[Azure Portal](portal.azure.com)で新しいアプリ登録を作成することです。これは、アプリケーションとAzure Active Directory/Entra ID間のOAuth統合に使用されます。アプリケーションに適切な名前を付け、リダイレクトURIは後で設定するため今は空白のままにして、保存してください。

![gptactions_outlook_registerapplication.png](../../../images/gptactions_outlook_registerapplication.png)

2. <b>証明書とシークレット</b>: 次に、GPTとAzure間の安全な通信を提供するためのクライアントシークレットを生成する必要があります。アプリ登録内で、サイドバーの<b>証明書とシークレット</b>に移動してください

![gptactions_outlook_secrets.png](../../../images/gptactions_outlook_secrets.png)

「New client secret」をクリックし、希望する名前と有効期限で新しいクライアントシークレットを作成します。保存をクリックすると、GPT作成で使用するシークレットが提供されます。**Value**フィールドを必ず保存してください。これは作成時にのみ表示され、後で必要になります！

![gptactions_outlook_secretvalue.png](../../../images/gptactions_outlook_secretvalue.png)

3. <b>API権限</b>: 次のステップは、統合に対して特定の必要なアクションを実行するために必要なスコープを提供することです。
アプリ登録内で、サイドバーの<b>管理 > API権限</b>に移動してください。

![gptactions_outlook_permissions.png](../../../images/gptactions_outlook_permissions.png)

<b>Add a permission</b>をクリックし、開いたサイドメニューのオプションで<b>Microsoft graph > Delegated Permissions</b>を選択します。検索バーを使用して以下の権限を追加してください：
- Calendars.ReadWrite
- Mail.Read
- Mail.Send
- User.Read

![gptactions_outlook_permissionadd.png](../../../images/gptactions_outlook_permissionadd.png)

### ChatGPTにて

ChatGPTで、「Authentication」をクリックし、**「OAuth」**を選択してください。以下の情報を入力してください。

- **Client ID**: Azure登録アプリの概要ページの**Application (client) ID**の下に記載されている値
- **Client Secret**: **Azure Steps**のステップ2で保存したシークレットの**Value**

以下の2つの入力において、`<Tenant_ID>`を登録済みアプリの概要ページの**ディレクトリ (テナント) ID**に記載されている値に置き換えてください。

- **認証URL**: https://login.microsoftonline.com/<Tenant_ID>/oauth2/v2.0/authorize
- **トークンURL**: https://login.microsoftonline.com/<Tenant_ID>/oauth2/v2.0/token
- **スコープ**: https://graph.microsoft.com/User.Read https://graph.microsoft.com/Mail.Send https://graph.microsoft.com/Mail.Read https://graph.microsoft.com/Calendars.ReadWrite 
- **トークン交換方式**: デフォルト（POSTリクエスト）

### アクション後のステップ

ChatGPTで認証を設定したら、以下の手順に従ってアプリケーション内でActionを完成させてください。

- GPT ActionからコールバックURLをコピーする

![gptactions_outlook_callback.png](../../../images/gptactions_outlook_callback.png)

- Azureアプリで、**管理 > 認証**タブに移動し、**プラットフォームを追加**をクリックして、**Web**を選択し、**リダイレクトURI**の下にコールバックURLを追加します

![gptactions_outlook_redirectconfig.png](../../../images/gptactions_outlook_redirectconfig.png)
![gptactions_outlook_redirectinput.png](../../../images/gptactions_outlook_redirectinput.png)

### FAQ & トラブルシューティング

- **コールバックURLエラー**: ChatGPTでコールバックURLエラーが発生した場合は、コールバックURLの値を再確認してください。認証に加えられた変更によって、この値が時々変更される可能性があります。

*優先的に対応してほしい統合機能はありますか？統合機能にエラーがありますか？GitHubでPRやissueを作成していただければ、確認いたします。*